# Test Budget Tracker Element


In [1]:
import json
import boto3
import requests
from IPython.display import clear_output

In [2]:
import plaid
from plaid.api import plaid_api
from plaid.model.transactions_sync_request import TransactionsSyncRequest

## Get Parameters from AWS

In [13]:
ssm = boto3.client("ssm")
plaid_client_id = ssm.get_parameter(Name="plaid-client-id")["Parameter"]["Value"]
plaid_secret = ssm.get_parameter(Name="plaid-secret-dev")["Parameter"]["Value"]
plaid_access = ssm.get_parameter(Name="plaid-access-token-amex-dev")["Parameter"]["Value"]
next_cursor = ssm.get_parameter(Name="plaid-next-cursor-amex-dev")["Parameter"]["Value"]

## Plaid API
Access ID retrieved with Quickstart Link session

In [6]:
host = plaid.Environment.Development

configuration = plaid.Configuration(                               
    host=host,
    api_key={
        'clientId': plaid_client_id,                               
        'secret': plaid_secret,
        'plaidVersion': '2020-09-14'                               
    }   
)

api_client = plaid.ApiClient(configuration)                        
client = plaid_api.PlaidApi(api_client)

## Sync Transactions

In [11]:
cursor = "" # use this to start from beginning
# cursor = next_cursor

if cursor == "null":
    cursor = ""
    
has_more = True

added = []
modified = []
removed = []

while has_more:
    request = TransactionsSyncRequest(access_token=plaid_access, cursor=cursor)
    response = client.transactions_sync(request)
    
    added.extend(response["added"])
    modified.extend(response["modified"])
    removed.extend(response["removed"])
    
    has_more = response["has_more"]
    cursor = response["next_cursor"]
    
print("Sync'd {} additions, {} modifications, {} removals".format(len(added), len(modified), len(removed)))

Sync'd 470 additions, 0 modifications, 0 removals


In [13]:
rid = [r["transaction_id"] for r in removed]

for a in added:
    print(a)

{'account_id': '7LBYPwRr0ZI4nJB7VKa3cLQeX7XEAesA4kQmr',
 'account_owner': 'KELLY DAVISON 1008',
 'amount': 95.93,
 'authorized_date': datetime.date(2023, 7, 6),
 'authorized_datetime': None,
 'category': ['Shops', 'Supermarkets and Groceries'],
 'category_id': '19047000',
 'check_number': None,
 'date': datetime.date(2023, 7, 7),
 'datetime': None,
 'iso_currency_code': 'USD',
 'location': {'address': '1109 N Highland St',
              'city': 'Arlington',
              'country': None,
              'lat': None,
              'lon': None,
              'postal_code': None,
              'region': 'VA',
              'store_number': '640'},
 'merchant_name': "Trader Joe's",
 'name': "Trader Joe's",
 'payment_channel': 'in store',
 'payment_meta': {'by_order_of': None,
                  'payee': None,
                  'payer': None,
                  'payment_method': None,
                  'payment_processor': None,
                  'ppd_id': None,
                  'reason': None,

## Update Cursor in SSM
Set to null to start over

In [15]:
res = ssm.put_parameter(Name="plaid-next-cursor-amex-dev", Value=cursor, Overwrite=True)

In [4]:
next_cursor

'CAESJURqSjFhWE9WN0tDYXBlblJOYmt6SG1hRDNCODdCclVYNjZhNUoaBgi768qmBiIGCLvryqYGKgYIu+vKpgY='

## Write to JSON

In [19]:
added_dict = [str(t.to_dict()) for t in added]

with open("amex_dev_20230707.json", "w") as fptr:
    json.dump(added_dict, fptr, default=str)

## Write to DynamoDB

In [ ]:
ddb = boto3.client("dynamodb")
table = "budget-tracker-transactions"

Get by ID

In [ ]:
res = ddb.get_item(TableName=table, Key={"id":{"S": "OgYjNaBd3MFbpZyXYa6dCMXeZjdE0xI9Kx9Eg"}})


def response_to_item(response):
    r = {}
    
    for k, v in res["Item"].items():
        for vk in v:
            if vk == "N":
                r[k] = float(v[vk])
            else:
                r[k] = v[vk]
                
    return r

print(response_to_item(res))

In [ ]:
res

Edit Field

In [ ]:
res = ddb.update_item(TableName=table, Key={"id":{"S": "VwYZN1MqjbiRZYgKED3qcNyeA8wV14iKN4Kgn"}},
                      UpdateExpression="set description=:d",
                      ExpressionAttributeValues={":d": {"S":"Harris and Teeter"}})

In [ ]:
res = ddb.update_item(TableName=table, Key={"id":{"S": "testid003"}},
                      UpdateExpression="set #M=:m, category=:c",
                      ExpressionAttributeNames={"#M": "month"},
                      ExpressionAttributeValues={":m": {"S":"Jun"}, ":c": {"S": "Groceries"}})

Add item

Will raise ConditionCheckFailedException if id already exists

In [ ]:
res = ddb.put_item(TableName=table,
                   Item={"id": {"S": "testid003"},
                         "amount": {"N": "21.03"}},
                   ConditionExpression="attribute_not_exists(id)")

Query Secondary

In [ ]:
res = ddb.query(TableName=table, IndexName="month-category-index",
               KeyConditionExpression="#M = :m AND #C = :c",
               ExpressionAttributeNames={"#M": "month", "#C": "category"},
               ExpressionAttributeValues={":m": {"S": "May 23"}, ":c": {"S": "Groceries"}})

## Test Login to Cognito
Pass `response["AuthenticationResult"]["IdToken"]` to `request["Authorization"]`

In [16]:
cog = boto3.client("cognito-idp")

Log in

In [17]:
pwd = input()
clear_output()

In [20]:
resp = cog.initiate_auth(AuthFlow="USER_PASSWORD_AUTH",
                         AuthParameters={"USERNAME":"peter","PASSWORD":pwd},
                         ClientId="t4rpjmebifvs8adt4ktqbg16u")

idtoken = resp["AuthenticationResult"]["IdToken"]

Need to update initial password

In [ ]:
resp2 = cog.respond_to_auth_challenge(ClientId="t4rpjmebifvs8adt4ktqbg16u",
                                      ChallengeName="NEW_PASSWORD_REQUIRED",
                                      Session=resp["Session"],
                                      ChallengeResponses={"USERNAME":"peter", "NEW_PASSWORD":"***"})

Accidentally set pwd wrong, so change it

In [ ]:
resp3 = cog.change_password(PreviousPassword="***",
                            ProposedPassword="***",
                            AccessToken=resp2["AuthenticationResult"]["AccessToken"])

### Add to DB with secured login

In [ ]:
t = [{"id": "a105", "date": "2023-01-01", "account": "test_acct", "description": "test 01", "amount": 1.99}]
url = "https://wsrxbgjqa1.execute-api.us-east-1.amazonaws.com/prod/add"
resp = requests.post(url, data=json.dumps(t), headers={"Authorization":idtoken})
resp.status_code

### Query DB with Login

In [ ]:
url = "https://wsrxbgjqa1.execute-api.us-east-1.amazonaws.com/prod/query"
params = {"type": "month-category", "month": "May 23", "category": "Groceries"}

resp = requests.get(url, params=params, headers={"Authorization":idtoken})
transactions = resp.json()

By ID

In [ ]:
url = "https://wsrxbgjqa1.execute-api.us-east-1.amazonaws.com/prod/query"
params = {"type": "id", "id": "a101"}

resp = requests.get(url, params=params, headers={"Authorization":idtoken})
transaction = resp.json()

### Edit DB with Login

In [ ]:
url = "https://wsrxbgjqa1.execute-api.us-east-1.amazonaws.com/prod/edit"
params = {"id": "a102", "amount": 2.65, "category":"Groceries", "checked": "True"}
# params = {"id_list": ["a100", "a101"]}
resp = requests.put(url, params=params, headers={"Authorization":idtoken})
resp.status_code

### Delete by ID with Login

In [29]:
url = "https://wsrxbgjqa1.execute-api.us-east-1.amazonaws.com/prod/delete"
params = {"id":"testid2"}
resp = requests.delete(url, params=params, headers={"Authorization": idtoken})
resp.status_code

200

## Bulk Write/Delete to DB using SNS
Do actual sync by running `python lambda_function.py amex /path/to/file.json` from `lambdas/sync_transactions` after copying `budget_tracker_common.py` into directory

In [27]:
topic = "arn:aws:sns:us-east-1:014374244911:budget-tracker-add-topic"

sns = boto3.client("sns")

msg = [{"id": "testid1", "date": "2023-01-01", "account": "test_acct", "description": "test 01", "amount": 1.99},
       {"id": "testid2", "date": "2023-01-01", "account": "test_acct", "description": "test 01", "amount": 1.99},
       {"id": "testid3", "date": "2023-01-01", "account": "test_acct", "description": "test 01", "amount": 1.99}]

res = sns.publish(TopicArn=topic, Message=json.dumps(msg))

In [28]:
topic = "arn:aws:sns:us-east-1:014374244911:budget-tracker-delete-topic"

sns = boto3.client("sns")

msg = [{"id": "testid1"},
       {"id": "testid3"}]

res = sns.publish(TopicArn=topic, Message=json.dumps(msg))

In [24]:
res

{'MessageId': '233b7688-810c-5006-8170-2bc15fa011e4',
 'ResponseMetadata': {'RequestId': '5a80adbe-e827-5a68-b6dd-f0f7368b6b8a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5a80adbe-e827-5a68-b6dd-f0f7368b6b8a',
   'content-type': 'text/xml',
   'content-length': '294',
   'date': 'Sun, 06 Aug 2023 17:46:19 GMT'},
  'RetryAttempts': 0}}

In [ ]:
with open("amex_dev_20230522.json", "r") as fptr:
    data = json.load(fptr)